In [10]:
import numpy as np
import pandas as pd

# =========================
# 0) 데이터 로드/전처리
# =========================
file_name = "mt-10yr0901-new.csv"
df = pd.read_csv(file_name, header=None)
df.columns = [
    "Date", "Open", "High", "Low", "Close",
    "Index1", "Index2", "Index3", "Index4", "Index5",
    "Index6", "Index7", "Index8", "Index9", "Index10", "Index11"
]
df["Date"] = pd.to_datetime(df["Date"]).dt.normalize()

def calculate_pct_change(df, col_name_current, col_name_previous):
    return (df[col_name_current] / df[col_name_previous].shift(1) - 1) * 100

# (시가 / 전일 종가 - 1) * 100
df["Open_Pct_Change"] = calculate_pct_change(df, "Open", "Close")
# Index6의 전일 대비 변동률
df["Index6_Pct_Change"] = calculate_pct_change(df, "Index6", "Index6")

# =========================
# 1) 벡터화 준비
# =========================
index_col = "Index6_Pct_Change"
open_pct_col = "Open_Pct_Change"
open_col = "Open"
rising_rate = 2.460
falling_rate = 2.520

a = df[index_col].to_numpy(dtype=float)            # 인덱스 변동률
opct = df[open_pct_col].to_numpy(dtype=float)      # 시가 변동률
openv = df[open_col].to_numpy(dtype=float)
closev = df["Close"].to_numpy(dtype=float)

close_minus_open = closev - openv
open_minus_close = openv - closev

# 미리 계산해두는 비교 기준 (각 시나리오에서 사용)
adj_rising = a * rising_rate
adj_falling = a * falling_rate

# =========================
# 2) 각 필터 값에 대한 수익을 벡터로 계산하는 함수들
# =========================
def profit_rising_min(filter_value: float) -> float:
    mask = (a > 0) & (a > filter_value)
    cmp = opct - adj_rising
    contrib = np.where(cmp > 0, close_minus_open, np.where(cmp < 0, open_minus_close, 0.0))
    return float(np.sum(np.where(mask, contrib, 0.0)))

def profit_rising_max(filter_value: float) -> float:
    mask = (a > 0) & (a < filter_value)
    cmp = opct - adj_rising
    contrib = np.where(cmp > 0, close_minus_open, np.where(cmp < 0, open_minus_close, 0.0))
    return float(np.sum(np.where(mask, contrib, 0.0)))

def profit_falling_min(filter_value: float) -> float:
    mask = (a < 0) & (a > filter_value)
    cmp = opct - adj_falling
    contrib = np.where(cmp < 0, open_minus_close, np.where(cmp > 0, close_minus_open, 0.0))
    return float(np.sum(np.where(mask, contrib, 0.0)))

def profit_falling_max(filter_value: float) -> float:
    mask = (a < 0) & (a < filter_value)
    cmp = opct - adj_falling
    contrib = np.where(cmp < 0, open_minus_close, np.where(cmp > 0, close_minus_open, 0.0))
    return float(np.sum(np.where(mask, contrib, 0.0)))

# =========================
# 3) 최적 필터 탐색 (간극 0.01 유지)
# =========================
def argmax_with_range(fn, start, stop, step):
    best_val = -np.inf
    best_arg = None
    vals = np.arange(start, stop, step)
    for v in vals:
        p = fn(float(v))
        if p > best_val:
            best_val = p
            best_arg = float(v)
    return best_arg, best_val

# 기존 범위/간극 그대로
rising_filter_min, profit_r_min = argmax_with_range(profit_rising_min, 0.0, 0.5, 0.005)
rising_filter_max, profit_r_max = argmax_with_range(profit_rising_max, 0.5, 1.5, 0.005)
falling_filter_min, profit_f_min = argmax_with_range(profit_falling_min, -1.5, -0.5, 0.005)
falling_filter_max, profit_f_max = argmax_with_range(profit_falling_max, -0.5, 0.0, 0.005)

# =========================
# 4) 최적 필터값으로 다시 각 수익 계산 (검증용)
# =========================
final_profit_r_min = profit_rising_min(rising_filter_min)
final_profit_r_max = profit_rising_max(rising_filter_max)
final_profit_f_min = profit_falling_min(falling_filter_min)
final_profit_f_max = profit_falling_max(falling_filter_max)

# 최적화된 수익 계산
total_profit = final_profit_r_min + final_profit_r_max + final_profit_f_min + final_profit_f_max

# 출력
print(f"상승 상황 rising_filter_min 수익: {final_profit_r_min:.2f}")
print(f"상승 상황 rising_filter_max 수익: {final_profit_r_max:.2f}")
print(f"하락 상황 falling_filter_min 수익: {final_profit_f_min:.2f}")
print(f"하락 상황 falling_filter_max 수익: {final_profit_f_max:.2f}")

# 최적 필터값과 총 수익
print(f"최적 필터값 -> "
      f"Rising Min = {rising_filter_min:.2f}, "
      f"Rising Max = {rising_filter_max:.2f}, "
      f"Falling Min = {falling_filter_min:.2f}, "
      f"Falling Max = {falling_filter_max:.2f}")





상승 상황 rising_filter_min 수익: 53.79
상승 상황 rising_filter_max 수익: 49.33
하락 상황 falling_filter_min 수익: 71.17
하락 상황 falling_filter_max 수익: 70.67
최적 필터값 -> Rising Min = 0.04, Rising Max = 1.31, Falling Min = -1.05, Falling Max = -0.00


In [ ]:
import numpy as np
import pandas as pd

# =========================
# 0) 데이터 로드/전처리
# =========================
file_name = "mt-10yr0901-new.csv"
df = pd.read_csv(file_name, header=None)
df.columns = [
    "Date", "Open", "High", "Low", "Close",
    "Index1", "Index2", "Index3", "Index4", "Index5",
    "Index6", "Index7", "Index8", "Index9", "Index10", "Index11"
]
df["Date"] = pd.to_datetime(df["Date"]).dt.normalize()

def calculate_pct_change(df, col_name_current, col_name_previous):
    return (df[col_name_current] / df[col_name_previous].shift(1) - 1) * 100

# (시가 / 전일 종가 - 1) * 100
df["Open_Pct_Change"] = calculate_pct_change(df, "Open", "Close")
# Index6의 전일 대비 변동률
df["Index6_Pct_Change"] = calculate_pct_change(df, "Index6", "Index6")

# =========================
# 1) 벡터화 준비
# =========================
index_col = "Index6_Pct_Change"
open_pct_col = "Open_Pct_Change"
open_col = "Open"
rising_rate = 2.460
falling_rate = 2.520

a = df[index_col].to_numpy(dtype=float)            # 인덱스 변동률
opct = df[open_pct_col].to_numpy(dtype=float)      # 시가 변동률
openv = df[open_col].to_numpy(dtype=float)
closev = df["Close"].to_numpy(dtype=float)

close_minus_open = closev - openv
open_minus_close = openv - closev

# 미리 계산해두는 비교 기준 (각 시나리오에서 사용)
adj_rising = a * rising_rate
adj_falling = a * falling_rate

# =========================
# 2) 각 필터 값에 대한 수익을 벡터로 계산하는 함수들
# =========================
def profit_rising_min(filter_value: float) -> float:
    mask = (a > 0) & (a > filter_value)
    cmp = opct - adj_rising
    contrib = np.where(cmp > 0, close_minus_open, np.where(cmp < 0, open_minus_close, 0.0))
    return float(np.sum(np.where(mask, contrib, 0.0)))

def profit_rising_max(filter_value: float) -> float:
    mask = (a > 0) & (a < filter_value)
    cmp = opct - adj_rising
    contrib = np.where(cmp > 0, close_minus_open, np.where(cmp < 0, open_minus_close, 0.0))
    return float(np.sum(np.where(mask, contrib, 0.0)))

def profit_falling_min(filter_value: float) -> float:
    mask = (a < 0) & (a > filter_value)
    cmp = opct - adj_falling
    contrib = np.where(cmp < 0, open_minus_close, np.where(cmp > 0, close_minus_open, 0.0))
    return float(np.sum(np.where(mask, contrib, 0.0)))

def profit_falling_max(filter_value: float) -> float:
    mask = (a < 0) & (a < filter_value)
    cmp = opct - adj_falling
    contrib = np.where(cmp < 0, open_minus_close, np.where(cmp > 0, close_minus_open, 0.0))
    return float(np.sum(np.where(mask, contrib, 0.0)))

# =========================
# 3) 최적 필터 탐색 (간극 0.01 유지)
# =========================
def argmax_with_range(fn, start, stop, step):
    best_val = -np.inf
    best_arg = None
    vals = np.arange(start, stop, step)
    for v in vals:
        p = fn(float(v))
        if p > best_val:
            best_val = p
            best_arg = float(v)
    return best_arg, best_val

# 기존 범위/간극 그대로
rising_filter_min, profit_r_min = argmax_with_range(profit_rising_min, 0.0, 0.5, 0.005)
rising_filter_max, profit_r_max = argmax_with_range(profit_rising_max, 0.5, 1.5, 0.005)
falling_filter_min, profit_f_min = argmax_with_range(profit_falling_min, -1.5, -0.5, 0.005)
falling_filter_max, profit_f_max = argmax_with_range(profit_falling_max, -0.5, 0.0, 0.005)

# =========================
# 4) 최적 필터값으로 다시 각 수익 계산 (검증용)
# =========================
final_profit_r_min = profit_rising_min(rising_filter_min)
final_profit_r_max = profit_rising_max(rising_filter_max)
final_profit_f_min = profit_falling_min(falling_filter_min)
final_profit_f_max = profit_falling_max(falling_filter_max)

# 최적화된 수익 계산
total_profit = final_profit_r_min + final_profit_r_max + final_profit_f_min + final_profit_f_max

# 출력
print(f"상승 상황 rising_filter_min 수익: {final_profit_r_min:.2f}")
print(f"상승 상황 rising_filter_max 수익: {final_profit_r_max:.2f}")
print(f"하락 상황 falling_filter_min 수익: {final_profit_f_min:.2f}")
print(f"하락 상황 falling_filter_max 수익: {final_profit_f_max:.2f}")

# 최적 필터값과 총 수익
print(f"최적 필터값 -> "
      f"Rising Min = {rising_filter_min:.3f}, "
      f"Rising Max = {rising_filter_max:.3f}, "
      f"Falling Min = {falling_filter_min:.3f}, "
      f"Falling Max = {falling_filter_max:.3f}")

# =========================
# 5) Rising Min 수익이 높은 순서대로 10개 출력
# =========================
# `Rising Min` 값에 대한 수익을 계산하고, 내림차순으로 정렬하여 상위 10개 출력
rising_min_values = np.arange(0.0, 0.5, 0.005)
profits_rising_min = [profit_rising_min(val) for val in rising_min_values]

# 수익을 내림차순으로 정렬하고 상위 10개 값 추출
sorted_indices = np.argsort(profits_rising_min)[::-1]  # 내림차순 정렬
top_10_rising_min = [(rising_min_values[i], profits_rising_min[i]) for i in sorted_indices[:10]]

print("\nTop 10 Rising Min 수익:")
for idx, (rmin, profit) in enumerate(top_10_rising_min, 1):
    print(f"{idx}. Rising Min = {rmin:.3f}, 수익 = {profit:.2f}")


상승 상황 rising_filter_min 수익: 53.79
상승 상황 rising_filter_max 수익: 49.33
하락 상황 falling_filter_min 수익: 71.17
하락 상황 falling_filter_max 수익: 70.67
최적 필터값 -> Rising Min = 0.0450, Rising Max = 1.3050, Falling Min = -1.0500, Falling Max = -0.0050

Top 10 Rising Min 수익:
1. Rising Min = 0.045, 수익 = 53.79
2. Rising Min = 0.025, 수익 = 53.33
3. Rising Min = 0.035, 수익 = 53.00
4. Rising Min = 0.020, 수익 = 52.97
5. Rising Min = 0.095, 수익 = 52.39
6. Rising Min = 0.030, 수익 = 51.93
7. Rising Min = 0.100, 수익 = 51.76
8. Rising Min = 0.110, 수익 = 51.73
9. Rising Min = 0.105, 수익 = 51.64
10. Rising Min = 0.015, 수익 = 51.60


In [27]:
import numpy as np
import pandas as pd

# =========================
# 0) 데이터 로드/전처리
# =========================
file_name = "mt-10yr0901-new.csv"
df = pd.read_csv(file_name, header=None)
df.columns = [
    "Date", "Open", "High", "Low", "Close",
    "Index1", "Index2", "Index3", "Index4", "Index5",
    "Index6", "Index7", "Index8", "Index9", "Index10", "Index11"
]
df["Date"] = pd.to_datetime(df["Date"]).dt.normalize()

def calculate_pct_change(df, col_name_current, col_name_previous):
    return (df[col_name_current] / df[col_name_previous].shift(1) - 1) * 100

# (시가 / 전일 종가 - 1) * 100
df["Open_Pct_Change"] = calculate_pct_change(df, "Open", "Close")
# Index6의 전일 대비 변동률
df["Index6_Pct_Change"] = calculate_pct_change(df, "Index6", "Index6")

# =========================
# 1) 벡터화 준비
# =========================
index_col = "Index6_Pct_Change"
open_pct_col = "Open_Pct_Change"
open_col = "Open"
rising_rate = 2.460
falling_rate = 2.520

a = df[index_col].to_numpy(dtype=float)            # 인덱스 변동률
opct = df[open_pct_col].to_numpy(dtype=float)      # 시가 변동률
openv = df[open_col].to_numpy(dtype=float)
closev = df["Close"].to_numpy(dtype=float)

close_minus_open = closev - openv
open_minus_close = openv - closev

# 미리 계산해두는 비교 기준 (각 시나리오에서 사용)
adj_rising = a * rising_rate
adj_falling = a * falling_rate

# =========================
# 2) 각 필터 값에 대한 수익을 벡터로 계산하는 함수들
# =========================
def profit_rising_min(filter_value: float) -> float:
    mask = (a > 0) & (a > filter_value)
    cmp = opct - adj_rising
    contrib = np.where(cmp > 0, close_minus_open, np.where(cmp < 0, open_minus_close, 0.0))
    return float(np.sum(np.where(mask, contrib, 0.0)))

def profit_rising_max(filter_value: float) -> float:
    mask = (a > 0) & (a < filter_value)
    cmp = opct - adj_rising
    contrib = np.where(cmp > 0, close_minus_open, np.where(cmp < 0, open_minus_close, 0.0))
    return float(np.sum(np.where(mask, contrib, 0.0)))

def profit_falling_min(filter_value: float) -> float:
    mask = (a < 0) & (a > filter_value)
    cmp = opct - adj_falling
    contrib = np.where(cmp < 0, open_minus_close, np.where(cmp > 0, close_minus_open, 0.0))
    return float(np.sum(np.where(mask, contrib, 0.0)))

def profit_falling_max(filter_value: float) -> float:
    mask = (a < 0) & (a < filter_value)
    cmp = opct - adj_falling
    contrib = np.where(cmp < 0, open_minus_close, np.where(cmp > 0, close_minus_open, 0.0))
    return float(np.sum(np.where(mask, contrib, 0.0)))

# =========================
# 3) 최적 필터 탐색 (간극 0.01 유지)
# =========================
def argmax_with_range(fn, start, stop, step):
    best_val = -np.inf
    best_arg = None
    vals = np.arange(start, stop, step)
    for v in vals:
        p = fn(float(v))
        if p > best_val:
            best_val = p
            best_arg = float(v)
    return best_arg, best_val

# 기존 범위/간극 그대로
rising_filter_min, profit_r_min = argmax_with_range(profit_rising_min, 0.0, 0.5, 0.005)
rising_filter_max, profit_r_max = argmax_with_range(profit_rising_max, 0.5, 1.5, 0.005)
falling_filter_min, profit_f_min = argmax_with_range(profit_falling_min, -1.5, -0.5, 0.005)
falling_filter_max, profit_f_max = argmax_with_range(profit_falling_max, -0.5, 0.0, 0.005)

# =========================
# 4) 최적 필터값으로 다시 각 수익 계산 (검증용)
# =========================
final_profit_r_min = profit_rising_min(rising_filter_min)
final_profit_r_max = profit_rising_max(rising_filter_max)
final_profit_f_min = profit_falling_min(falling_filter_min)
final_profit_f_max = profit_falling_max(falling_filter_max)

# 최적화된 수익 계산
total_profit = final_profit_r_min + final_profit_r_max + final_profit_f_min + final_profit_f_max

# 출력
print(f"상승 상황 rising_filter_min 수익: {final_profit_r_min:.2f}")
print(f"상승 상황 rising_filter_max 수익: {final_profit_r_max:.2f}")
print(f"하락 상황 falling_filter_min 수익: {final_profit_f_min:.2f}")
print(f"하락 상황 falling_filter_max 수익: {final_profit_f_max:.2f}")

# 최적 필터값과 총 수익
print(f"최적 필터값 -> "
      f"Rising Min = {rising_filter_min:.3f}, "
      f"Rising Max = {rising_filter_max:.3f}, "
      f"Falling Min = {falling_filter_min:.3f}, "
      f"Falling Max = {falling_filter_max:.3f}")

# =========================
# 5) Rising Min, Rising Max, Falling Min, Falling Max Top 10 출력
# =========================

# `Rising Min`, `Rising Max`, `Falling Min`, `Falling Max` 값에 대한 수익을 계산하고, 내림차순으로 정렬하여 상위 10개 출력
rising_min_values = np.arange(0.0, 0.5, 0.005)
profits_rising_min = [profit_rising_min(val) for val in rising_min_values]
sorted_rising_min_indices = np.argsort(profits_rising_min)[::-1]  # 내림차순 정렬
top_10_rising_min = [(rising_min_values[i], profits_rising_min[i]) for i in sorted_rising_min_indices[:10]]

rising_max_values = np.arange(0.5, 1.5, 0.005)
profits_rising_max = [profit_rising_max(val) for val in rising_max_values]
sorted_rising_max_indices = np.argsort(profits_rising_min)[::-1]  # 내림차순 정렬
top_10_rising_max = [(rising_max_values[i], profits_rising_max[i]) for i in sorted_rising_max_indices[:10]]

falling_min_values = np.arange(-1.5, -0.5, 0.005)
profits_falling_min = [profit_falling_min(val) for val in falling_min_values]
sorted_falling_min_indices = np.argsort(profits_falling_min)[::-1]  # 내림차순 정렬
top_10_falling_min = [(falling_min_values[i], profits_falling_min[i]) for i in sorted_falling_min_indices[:10]]

falling_max_values = np.arange(-0.5, 0.0, 0.005)
profits_falling_max = [profit_falling_max(val) for val in falling_max_values]
sorted_falling_max_indices = np.argsort(profits_falling_max)[::-1]  # 내림차순 정렬
top_10_falling_max = [(falling_max_values[i], profits_falling_max[i]) for i in sorted_falling_max_indices[:10]]

# =========================
# 6) 출력
# =========================
print("\nTop 10 Rising Min 수익:")
for idx, (rmin, profit) in enumerate(top_10_rising_min, 1):
    print(f"{idx}. Rising Min = {rmin:.3f}, 수익 = {profit:.2f}")

print("\nTop 10 Rising Max 수익:")
for idx, (rmax, profit) in enumerate(top_10_rising_max, 1):
    print(f"{idx}. Rising Max = {rmax:.3f}, 수익 = {profit:.2f}")

print("\nTop 10 Falling Min 수익:")
for idx, (fmin, profit) in enumerate(top_10_falling_min, 1):
    print(f"{idx}. Falling Min = {fmin:.3f}, 수익 = {profit:.2f}")

print("\nTop 10 Falling Max 수익:")
for idx, (fmax, profit) in enumerate(top_10_falling_max, 1):
    print(f"{idx}. Falling Max = {fmax:.3f}, 수익 = {profit:.2f}")

상승 상황 rising_filter_min 수익: 53.79
상승 상황 rising_filter_max 수익: 49.33
하락 상황 falling_filter_min 수익: 71.17
하락 상황 falling_filter_max 수익: 70.67
최적 필터값 -> Rising Min = 0.045, Rising Max = 1.305, Falling Min = -1.050, Falling Max = -0.005

Top 10 Rising Min 수익:
1. Rising Min = 0.045, 수익 = 53.79
2. Rising Min = 0.025, 수익 = 53.33
3. Rising Min = 0.035, 수익 = 53.00
4. Rising Min = 0.020, 수익 = 52.97
5. Rising Min = 0.095, 수익 = 52.39
6. Rising Min = 0.030, 수익 = 51.93
7. Rising Min = 0.100, 수익 = 51.76
8. Rising Min = 0.110, 수익 = 51.73
9. Rising Min = 0.105, 수익 = 51.64
10. Rising Min = 0.015, 수익 = 51.60

Top 10 Rising Max 수익:
1. Rising Max = 0.545, 수익 = 44.51
2. Rising Max = 0.525, 수익 = 42.27
3. Rising Max = 0.535, 수익 = 42.90
4. Rising Max = 0.520, 수익 = 41.83
5. Rising Max = 0.595, 수익 = 45.75
6. Rising Max = 0.530, 수익 = 42.90
7. Rising Max = 0.600, 수익 = 45.75
8. Rising Max = 0.610, 수익 = 46.45
9. Rising Max = 0.605, 수익 = 46.45
10. Rising Max = 0.515, 수익 = 41.46

Top 10 Falling Min 수익:
1. Falling Min = 